In [1]:
!pip install fuzzywuzzy


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
from fuzzywuzzy import process, fuzz

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
## Data File Path
FEATURES_PARQUET  = '../../data/raw/echonest.parquet'
TRACKS_PARQUET = '../../data/raw/tracks.parquet'

In [4]:
tracks = pd.read_parquet(TRACKS_PARQUET)
#tracks.columns
tracks.head()

album                                                      
         comments         date_created        date_released engineer   
track_id                                                               
2               0  2008-11-26 01:44:45  2009-01-05 00:00:00     None  \
3               0  2008-11-26 01:44:45  2009-01-05 00:00:00     None   
5               0  2008-11-26 01:44:45  2009-01-05 00:00:00     None   
10              0  2008-11-26 01:45:08  2008-02-06 00:00:00     None   
20              0  2008-11-26 01:45:05  2009-01-06 00:00:00     None   

                                                                           
         favorites id                                information listens   
track_id                                                                   
2                4  1                                    <p></p>    6073  \
3                4  1                                    <p></p>    6073   
5                4  1                                    <p></p>    6073   
10               4  6                                       None   47632   
20               2  4  <p> "spiritual songs" from Nicky Cook</p>    2710   

                        ...       track                          
         producer tags  ... information interest language_code   
track_id                ...                                      
2            None   []  ...        None     4656            en  \
3            None   []  ...        None     1470            en   
5            None   []  ...        None     1933            en   
10           None   []  ...        None    54881            en   
20           None   []  ...        None      978            en   

                                                                               
                                                    license listens lyricist   
track_id                                                                       
2         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293     None  \
3         Attribution-NonCommercial-ShareAlike 3.0 Inter...     514     None   
5         Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151     None   
10        Attribution-NonCommercial-NoDerivatives (aka M...   50135     None   
20        Attribution-NonCommercial-NoDerivatives (aka M...     361     None   

                                                 
         number publisher tags            title  
track_id                                         
2             3      None   []             Food  
3             4      None   []     Electric Ave  
5             6      None   []       This World  
10            1      None   []          Freeway  
20            3      None   []  Spiritual Level  

[5 rows x 52 columns]

In [5]:
features = pd.read_parquet(FEATURES_PARQUET)
features.echonest.metadata.head()

,album_date,album_name,artist_latitude,artist_location,artist_longitude,artist_name,release
track_id,,,,,,,
2,None,None,32.6783,"Georgia, US",-83.2230,AWOL,AWOL - A Way Of Life
3,None,None,32.6783,"Georgia, US",-83.2230,AWOL,AWOL - A Way Of Life
5,None,None,32.6783,"Georgia, US",-83.2230,AWOL,AWOL - A Way Of Life
10,2008-03-11,Constant Hitmaker,39.9523,"Philadelphia, PA, US",-75.1624,Kurt Vile,Constant Hitmaker
134,None,None,32.6783,"Georgia, US",-83.2230,AWOL,AWOL - A Way Of Life


In [6]:
track_features = pd.merge(features.echonest,tracks,on="track_id",how="left") # merge also on aritst name and date created or smth like that
track_features.shape

(13129, 301)

# Load Tags Data

In [7]:
MUSIC_TAGS_PARQUET_FILE='../../data/raw/final_data_v2.parquet'
music_tags_df = pd.read_parquet(MUSIC_TAGS_PARQUET_FILE, engine='fastparquet')
music_tags_df

,tid,title,artist,tag
0,TRCCCSL128F4260C90,At My Favorite Place,Ariel Kill Him,Akinyele
1,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,chillout
2,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,dance
3,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,downbeat
4,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,ion b chill station
...,...,...,...,...
7659938,TRVVVTG128F9305E3C,Mozambique,Les Baxter,Mozambique
7659939,TRVVVTG128F9305E3C,Mozambique,Les Baxter,BO
7659940,TRVVVTG128F9305E3C,Mozambique,Les Baxter,3 stars
7659941,TRVVVTG128F9305E3C,Mozambique,Les Baxter,zart


In [110]:
# tag = "chill"
# mask = music_tags_df["tag"].str.match(tag,case=False)
# masked_tagged_titles = music_tags_df[mask]

In [112]:
## Comparing all tracks will take too long we need to scope tracks with tags

# unique_tracks_series1 = tracks["track"]["title"].unique()
# unique_tracks_df = tracks[[("track","title")]] 
# unique_tracks_df.shape

# unique_tracks_series2 = masked_tagged_titles["title"].unique()
# #unique_tracks_series2.shape

# unique_tracks_series1_proc = unique_tracks_series1.copy()
# unique_tracks_series2_proc = unique_tracks_series2.copy()

# #print(len(unique_tracks_series1_proc))

# candidate_titles=[]
# similarity=[]
# counter = 0
# for title1 in unique_tracks_series1_proc:
#     for title2 in unique_tracks_series2:
#         ratio = fuzz.ratio( title1, title2)
#         if ratio >= 95:
#             print(title1,title2,ratio)
#             candidate_titles.append([title1,title2])
#             similarity.append(ratio)
#     counter+=1
#     print(counter)


In [8]:
music_tags_df.head()

,tid,title,artist,tag
0,TRCCCSL128F4260C90,At My Favorite Place,Ariel Kill Him,Akinyele
1,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,chillout
2,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,dance
3,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,downbeat
4,TRCCCSK128F92EE3B2,Endangered Species,Nickodemus,ion b chill station


# Merge Two Datasets

In [9]:
track_features.head()
track_features_enhanced = track_features.reset_index(level=["track_id"])
track_features_enhanced.columns

MultiIndex([(      'track_id',                 ''),
            ('audio_features',     'acousticness'),
            ('audio_features',     'danceability'),
            ('audio_features',           'energy'),
            ('audio_features', 'instrumentalness'),
            ('audio_features',         'liveness'),
            ('audio_features',      'speechiness'),
            ('audio_features',            'tempo'),
            ('audio_features',          'valence'),
            (      'metadata',       'album_date'),
            ...
            (         'track',      'information'),
            (         'track',         'interest'),
            (         'track',    'language_code'),
            (         'track',          'license'),
            (         'track',          'listens'),
            (         'track',         'lyricist'),
            (         'track',           'number'),
            (         'track',        'publisher'),
            (         'track',             'tags

In [10]:
multi_indices = [("track",col) for col in  music_tags_df.columns.to_list()]
multi_indices
music_tags_df_enhanced = music_tags_df.copy()
music_tags_df_enhanced.columns = pd.MultiIndex.from_tuples(multi_indices)
music_tags_df_enhanced.columns
#music_tags_df_enhanced.columns

MultiIndex([('track',    'tid'),
            ('track',  'title'),
            ('track', 'artist'),
            ('track',    'tag')],
           )

In [11]:
merged_df = pd.merge(track_features_enhanced,music_tags_df_enhanced,how="inner")
#merged_df.title.unique()
merged_df.head()

track_id audio_features                                                     
             acousticness danceability    energy instrumentalness  liveness   
0        2       0.416675     0.675894  0.634476         0.010628  0.177647  \
1        2       0.416675     0.675894  0.634476         0.010628  0.177647   
2   107996       0.941069     0.712837  0.287110         0.020498  0.088466   
3   107996       0.941069     0.712837  0.287110         0.020498  0.088466   
4        5       0.043567     0.745566  0.701470         0.000697  0.373143   

                                   metadata  ...   
  speechiness    tempo   valence album_date  ...   
0    0.159310  165.922  0.576661       None  ...  \
1    0.159310  165.922  0.576661       None  ...   
2    0.038736  136.097  0.338812       None  ...   
3    0.038736  136.097  0.338812       None  ...   
4    0.124595  100.260  0.621661       None  ...   

                                               track                           
                                             license listens lyricist number   
0  Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293     None      3  \
1  Attribution-NonCommercial-ShareAlike 3.0 Inter...    1293     None      3   
2  Creative Commons Attribution-NonCommercial-NoD...    3889     None     12   
3  Creative Commons Attribution-NonCommercial-NoD...    3889     None     12   
4  Attribution-NonCommercial-ShareAlike 3.0 Inter...    1151     None      6   

                                                                             
  publisher tags       title                 tid                    artist   
0      None   []        Food  TREKAFR128F933C68D  The Pursuit of Happiness  \
1      None   []        Food  TRTFHLV128F422C0D1                Zico Chain   
2      None   []        Food  TREKAFR128F933C68D  The Pursuit of Happiness   
3      None   []        Food  TRTFHLV128F422C0D1                Zico Chain   
4      None   []  This World  TRCZHBU12903D0B40B           Mind Reflection   

                  
             tag  
0  the word food  
1    Post Grunge  
2  the word food  
3    Post Grunge  
4     THIS WORLD  

[5 rows x 305 columns]

In [20]:
merged_df.to_parquet('../../data/features/featuresv2.parquet')

## Sample Selection based on tags

In [12]:
tag = "running"
mask = merged_df.track.tag.str.match(tag,case=False)
masked_tagged_titles = merged_df[mask]
masked_tagged_titles

track_id audio_features                                           
                  acousticness danceability    energy instrumentalness   
22293      1171       0.343810     0.352761  0.942230         0.960118  \
33903      1519       0.232296     0.696068  0.823817         0.959458   
34003      1519       0.232296     0.696068  0.823817         0.959458   
34086      1519       0.232296     0.696068  0.823817         0.959458   
34175      1519       0.232296     0.696068  0.823817         0.959458   
...         ...            ...          ...       ...              ...   
536486    70238       0.000061     0.311664  0.969324         0.882616   
540432    73950       0.617472     0.405051  0.756096         0.000002   
552992    89460       0.072168     0.745554  0.586552         0.008015   
553120    89460       0.072168     0.745554  0.586552         0.008015   
566072   110743       0.000333     0.631644  0.593367         0.959862   

                                                  metadata  ...   
        liveness speechiness    tempo   valence album_date  ...   
22293   0.101288    0.040280  146.961  0.423637       None  ...  \
33903   0.108482    0.041674  100.309  0.921504       None  ...   
34003   0.108482    0.041674  100.309  0.921504       None  ...   
34086   0.108482    0.041674  100.309  0.921504       None  ...   
34175   0.108482    0.041674  100.309  0.921504       None  ...   
...          ...         ...      ...       ...        ...  ...   
536486  0.334249    0.075225   79.994  0.087549       None  ...   
540432  0.695170    0.040997  133.963  0.270675       None  ...   
552992  0.172923    0.275989  142.949  0.517008       None  ...   
553120  0.172923    0.275989  142.949  0.517008       None  ...   
566072  0.082872    0.041732   98.867  0.875357       None  ...   

                                                    track                    
                                                  license listens lyricist   
22293   Attribution-Noncommercial-Share Alike 3.0 Unit...     173     None  \
33903   Attribution-Noncommercial-Share Alike 3.0 Unit...      64     None   
34003   Attribution-Noncommercial-Share Alike 3.0 Unit...      64     None   
34086   Attribution-Noncommercial-Share Alike 3.0 Unit...      64     None   
34175   Attribution-Noncommercial-Share Alike 3.0 Unit...      64     None   
...                                                   ...     ...      ...   
536486  Attribution-Noncommercial-No Derivative Works ...    1055     None   
540432  Attribution-NonCommercial-ShareAlike 3.0 Inter...    3630     None   
552992  Attribution-NonCommercial-ShareAlike 3.0 Inter...    7201     None   
553120  Attribution-NonCommercial-ShareAlike 3.0 Inter...    7201     None   
566072               Attribution-NonCommercial-ShareAlike    2062     None   

                                                                 
       number publisher tags         title                 tid   
22293       8      None   []  Black Market  TRZSHFZ128F426902F  \
33903       2      None   []     Watch Out  TRAOXGW128F933AF95   
34003       2      None   []     Watch Out  TRNGZAX128F428109F   
34086       2      None   []     Watch Out  TREMFMF128F931BD1F   
34175       2      None   []     Watch Out  TRGBOEX128F425F3A0   
...       ...       ...  ...           ...                 ...   
536486      7      None   []       Ulysses  TREFXWL12903C95494   
540432      1      None   []       Holiday  TRGZHAP128F425AEF8   
552992      7      None   []         Jewel  TRABMWI128F428E582   
553120      7      None   []         Jewel  TRFAYVZ128F428E4C2   
566072      1      None   []      Textbook  TRDYJGC128F145255E   

                                         
                   artist           tag  
22293      Weather Report  running eyes  
33903        Alex Gaudino       running  
34003        Alex Gaudino       running  
34086        Alex Gaudino       running  
34175        Alex Gaudino       running  
... 

In [18]:
masked_tagged_titles.track.tag.value_counts()
#running, running mix, running music

tag
running                                       44
running eyes                                  10
running down hills and dancing in the rain     5
running out of the church                      4
running music                                  4
running in circles                             2
running away                                   2
running mix                                    1
running with scissors                          1
Running by NoDoubt                             1
Running Wild                                   1
Name: count, dtype: int64

In [19]:
unique_tags = merged_df.track.tag.value_counts()
unique_tags.head()

tag
rock           6643
pop            4741
alternative    4179
electronic     3511
indie          3168
Name: count, dtype: int64